In [ ]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [993 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,139 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,629 kB]
Get:13 http://security.ubuntu.com/ubuntu focal

In [ ]:
# !!! В Jupyter Notebook эту ячейку запускать не нужно !!!
!pip install imagemagick

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement imagemagick (from versions: none)
ERROR: No matching distribution found for imagemagick


In [ ]:
!pip install pdfminer.six
!pip install pdf2image
!pip install pathlib
!pip install pikepdf
!pip install poppler-utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pdfminer.pdfinterp      import PDFResourceManager, PDFPageInterpreter
from pdf2image               import convert_from_path
from PIL                     import Image, ImageDraw
from pdfminer.converter      import XMLConverter
from ctypes.util             import find_library
from typing                  import Container
from pdfminer.layout         import LAParams
from pdfminer.pdfpage        import PDFPage
from fnmatch                 import fnmatch
from io                      import BytesIO
from pathlib                 import Path


import matplotlib.pyplot     as plt
import xml.etree.ElementTree as ET
import numpy                 as np
import functools
import pikepdf
import pathlib
import os

In [ ]:
colors = ['blue', 'orange', 'green', 'red', 'purple',
          'brown', 'pink', 'gray',  'olive', 'cyan']

def convert_pdf(
    path: str,
    format: str = "xml",
    codec: str = "utf-8",
    password: str = "",
    maxpages: int = 0,
    caching: bool = True,
    pagenos: Container[int] = set(),
) -> str:
  """

  """
  rsrcmgr = PDFResourceManager()
  retstr = BytesIO()
  laparams = LAParams()
  device = XMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
  fp = open(path, "rb")
  interpreter = PDFPageInterpreter(rsrcmgr, device)
  for page in PDFPage.get_pages(fp,
                                pagenos,
                                maxpages=maxpages,
                                password=password,
                                caching=caching,
                                check_extractable=True):
      interpreter.process_page(page)
  text = retstr.getvalue().decode()
  fp.close()
  device.close()
  retstr.close()
  return text


def conv_pdf_to_xml(pdf_path: str,
                    separator: str = '\\'):
  """

  """
  path = pdf_path.rsplit(separator, 1)[0]
  pattern = "*.pdf"
  name = pdf_path.rsplit(separator)[-1]
  if fnmatch(name, pattern):

    # создаем xml
    name_xml = name.rsplit('.', 1)[0] + ".xml"
    xml_path = os.path.join(path, name_xml)  # Путь до xml файла
    open(xml_path, "w").close()

    # переводим pdf to xml
    text_output = convert_pdf(pdf_path)
    text_output = text_output + "</pages>"
    # print(xml_path)

    # сохраняем xml
    open(xml_path, "a", encoding="utf-8").write(text_output)
    # Возвращаем путь к xml-файлу и на папку с картинками
    return xml_path


def conv_pdf_to_png(pdf_path: str,
                    xml_path: str,
                    separator: str = '\\'):
  """

  """
  path = pdf_path.rsplit(separator, 1)[0]
  # создаем папку для изображений страниц
  dirname = 'pages ' + pdf_path.rsplit(separator, 1)[-1].split('.pdf')[0]
  if not os.path.exists(dirname):
    os.mkdir(dirname)

  # переводим pdf в png
  pages_png = convert_from_path(pdf_path)

  tree = ET.parse(xml_path)
  root = tree.getroot()
  pages_xml = root.findall('page')

  img_dir_path = os.path.join(path, dirname)
  for i in range(len(pages_png)):
    id = pages_xml[i].get('id')
    pages_png[i].save(os.path.join(img_dir_path, f'{id}.png'), 'PNG')
  return img_dir_path


# Функция, которая будет возвращать кусок xml-файла и соответствующую картинку
def get_page(page_id: int,
             xml_path: str,
             dir_path: str):
  """

  """
  tree = ET.parse(xml_path)
  root = tree.getroot()
  pages = root.findall('page')
  if page_id > len(pages):
    return 'page_id is bigger than pages quantity', 'page_id is bigger than pages quantity'
  page = pages[page_id - 1]
  img_path = os.path.join(dir_path, f'{page_id}.png')
  return page, img_path


def str_to_float(strs):
  """

  """
  floats = []
  for str_ in strs:
    floats.append(float(str_))
  return floats


def coords_xml_to_png(arr: list,
                      img_path: str,
                      page: ET.Element):
  """

  """
  image = Image.open(img_path)
  image_size_png = image.size
  image_size_xml = str_to_float(page.get('bbox').split(',')[2:])
  x_step = image_size_png[0] / image_size_xml[0]
  y_step = image_size_png[1] / image_size_xml[1]
  if len(np.array(arr).shape) == 1:
    return [arr[0] * x_step, \
            image_size_png[1] - arr[1] * y_step, \
            arr[2] * x_step, \
            image_size_png[1] - arr[3] * y_step]
  coords_png = []
  for points in arr:
    coords_png.append([points[0] * x_step, \
                       image_size_png[1] - points[1] * y_step, \
                       points[2] * x_step, \
                       image_size_png[1] - points[3] * y_step])
  return coords_png


def get_bboxes(page: ET.Element,
               name: str):
  """

  """
  bboxes = []
  for tag in page.findall(name):
    bbox_str = tag.get('bbox').split(',')
    bbox = str_to_float(bbox_str)
    bboxes.append(bbox)
  return bboxes


def table_edges(coords: list):
  """

  """
  return [coords[0][0], coords[0][1], coords[-1][2], coords[-1][3]]


def tables_on_the_page(coords: list):
  """

  """
  tables = []
  last_rect_num = -1
  for i in range(len(coords) - 3):
    if coords[i] == coords[i + 1] and coords[i + 2] == coords[i + 3]:
      last_rect_num = i + 2
      tables.append(coords[:i + 2])
  if last_rect_num >= 0:
    tables.append(coords[last_rect_num:])
  if len(tables) == 0:
    tables.append(coords)
    return tables
  return tables


def get_tables_bboxes(page, img_path):
  if len(get_bboxes(page, 'rect')) == 0:
    bboxes = get_bboxes(page, 'line')
    coords = coords_xml_to_png(bboxes, img_path, page)
    tables = [coords]
  else:
    bboxes = get_bboxes(page, 'rect')
    coords = coords_xml_to_png(bboxes, img_path, page)
    tables = tables_on_the_page(coords)
  return tables


def highlight_tables(page: ET.Element,
                     img_path: str,
                     disp: bool = True):
  """

  """
  tables = get_tables_bboxes(page, img_path)

  tbl_beg_wnh = []

  image = Image.open(img_path)
  draw_image = image.copy()
  draw = ImageDraw.Draw(draw_image)

  for i in range(len(tables)):
    edge = table_edges(tables[i])
    draw.rectangle(edge, outline=colors[i % len(colors)])
    tbl_beg_wnh.append([edge[0], edge[1], [edge[2] - edge[0], edge[3] - edge[1]]])
  if disp:
    display(draw_image)
  return tbl_beg_wnh


def replace_y0y1(elem):
  return [elem[0], elem[3], elem[2], elem[1]]


def highlight_cells(page: ET.Element,
                    img_path: str,
                    table,
                    disp: bool = True):
  """

  """
  # извлекаем блоки textline и сохраняет их в одном массиве
  if get_bboxes(page, 'rect') == []:
    tag = 'line'
  else:
    tag = 'rect'
  lor = coords_xml_to_png(get_bboxes(page, tag), img_path, page)
  textlines = []
  temp = []

  for textbox in page.findall('textbox'):
    temp.append(coords_xml_to_png(get_bboxes(textbox, 'textline'), \
                                  img_path, \
                                  page))

  for elem in temp:
    for coord in elem:
      textlines.append(coord)

  # разделяем блоки textline по таблицам
  tl_table = []
  tbl = table_edges(table)

  for textline in textlines:
    if tbl[0] <= textline[0] <= tbl[2] and \
       tbl[0] <= textline[2] <= tbl[2] and \
       tbl[1] <= textline[1] <= tbl[3] and \
       tbl[1] <= textline[3] <= tbl[3]:
      tl_table.append(textline)

  img = Image.open(img_path)
  draw = ImageDraw.Draw(img)

  cells = []
  for i in range(len(tl_table)):

    vlines_left = []
    vlines_right = []
    hlines_top = []
    hlines_bot = []

    for elem in table:
      if (elem[3] <= tl_table[i][1] <= elem[1] and \
          elem[3] <= tl_table[i][3] <= elem[1] and \
          tl_table[i][0] > elem[2]):
        vlines_left.append(elem)

      if (elem[3] <= tl_table[i][1] <= elem[1] and \
          elem[3] <= tl_table[i][3] <= elem[1] and \
          tl_table[i][2] < elem[0]):
        vlines_right.append(elem)

      if (elem[0] <= tl_table[i][0] <= elem[2] and \
          elem[0] <= tl_table[i][2] <= elem[2] and \
          tl_table[i][3] < elem[1]):
        hlines_top.append(elem)

      if (elem[0] <= tl_table[i][0] <= elem[2] and \
          elem[0] <= tl_table[i][2] <= elem[2] and \
          tl_table[i][1] > elem[3]):
        hlines_bot.append(elem)

    if vlines_left != [] and vlines_right != []:
      vlines_left_min_args = (np.argsort(np.sum((np.array(vlines_left) - np.array(tl_table[i]))[:, :2]**2, axis=1)))[0]
      vlines_right_min_args = (np.argsort(np.sum((np.array(vlines_right) - np.array(tl_table[i]))[:, :2]**2, axis=1)))[0]
      cell = table_edges([vlines_left[vlines_left_min_args], vlines_right[vlines_right_min_args]])
      if cell[3] < cell[1]:
        cell = replace_y0y1(cell)
      cells.append(cell)
      continue

    if hlines_top != [] and hlines_bot != []:
      hlines_top_min_args = (np.argsort(np.sum((np.array(hlines_top) - np.array(tl_table[i]))[:, :2]**2, axis=1)))[0]
      hlines_bot_min_args = (np.argsort(np.sum((np.array(hlines_bot) - np.array(tl_table[i]))[:, :2]**2, axis=1)))[0]
      cell = table_edges([hlines_top[hlines_top_min_args], hlines_bot[hlines_bot_min_args]])
      if cell[3] < cell[1]:
        cell = replace_y0y1(cell)
      cells.append(cell)
      continue

  cells = np.unique(np.array(cells), axis=0).tolist()
  # print('len(cells) =', len(cells))

  for i in range(len(cells)):
    draw.rectangle(cells[i], outline=colors[i % len(colors)])

  if disp:
    display(img)
  return (np.array(cells) - np.array([tbl[0], tbl[1], tbl[0], tbl[1]])).tolist()


def inside(area: list,
           text: list):
  """
  area - внутри чего должно находиться
  """
  res = area[0] <= text[0] <= area[2] and \
        area[0] <= text[2] <= area[2] and \
        area[1] <= text[1] <= area[3] and \
        area[1] <= text[3] <= area[3]
  return res


def highlight_text(area: list,
                   page: ET.Element,
                   img_path: str,
                   disp: bool = True):
  """

  """

  text_bboxes = []
  for textbox in page.findall('textbox'):
    for textline in textbox.findall('textline'):
      for text in textline.findall('text'):
        if text.get('bbox') != None:
          bbox_str = text.get('bbox').split(',')
          bbox = str_to_float(bbox_str)
          text_bboxes.append(bbox)
  texts = coords_xml_to_png(text_bboxes, img_path, page)

  img = Image.open(img_path)
  draw = ImageDraw.Draw(img)

  row = texts[0][0]
  j = 0
  for i in range(len(texts)):
    if inside(area, texts[i]):
      if texts[i][1] != row:
        j += 1
        row = texts[i][1]
      draw.rectangle(replace_y0y1(texts[i]), outline=colors[j % len(colors)])
      # print(round(texts[i][1]))
  if disp:
    display(img)

In [ ]:
file_name = 'test_doc_2.pdf'
separator='/'

path = str(Path(pathlib.Path.cwd(), file_name))
xml_path = conv_pdf_to_xml(path, separator=separator)
img_dir_path = conv_pdf_to_png(path, xml_path, separator=separator)
page, img_path = get_page(1, xml_path, img_dir_path)

PDFInfoNotInstalledError: ignored

In [ ]:
tables_ = highlight_tables(page, img_path, disp=False)

In [ ]:
tables = get_tables_bboxes(page, img_path)
tables_cells = []
for table in tables:
  tables_cells.append(highlight_cells(page, img_path, table, disp=True))

In [ ]:
tables_cells[0]

In [ ]:
area = [0, 0, 1000, 1000]
highlight_text(area, page, img_path)

In [ ]:
# собираем все текстлайны
ETtextlines = []
temp = []
for textbox in page.findall('textbox'):
  temp.append(textbox.findall('textline'))

for elem in temp:
  for ETtextline in elem:
    ETtextlines.append(ETtextline)

# высчитываем координаты ячеек таблицы
table_beg = np.array([tables_[0][0],
                      tables_[0][1],
                      tables_[0][0],
                      tables_[0][1]])
cells = (np.array(tables_cells[0]) + table_beg).tolist()

# собираем данные из ячеек в массив
tl_text = []
for cell in cells:
  tl_strokes = []
  for ETtextline in ETtextlines:
    textline = coords_xml_to_png(str_to_float(ETtextline.get('bbox') \
                                                            .split(',')), \
                                 img_path, \
                                 page)
    if inside(cell, textline):
      stroke = []
      for text in ETtextline.findall('text'):
        if text.text != '\n':
          stroke.append(text)
      tl_strokes.append(stroke)
  tl_text.append(tl_strokes)

In [ ]:
for i in range(len(tl_text)):
  print(tl_text[i])

NameError: ignored